In [1]:
%cd ../

/u02/thanh/workplace/plasma


In [2]:
import time

import plasma.parallel_processing.communicators as ppc
import plasma.parallel_processing.queues as ppq
from plasma.functional import partials, AutoPipe, LambdaPipe

/u02/thanh/workplace/plasma/.env/RnD/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compute(x):
    time.sleep(0.5)
    return f'compute: {x}'

def compute2(x):
    time.sleep(0.5)
    return f'compute2: {x + 'c'}'

class ComputeClass(AutoPipe):

    def __init__(self, hihi, haha):
        super().__init__()
    
    def run(self, tmp):
        return 'ComputeClass: ' + tmp + 'd'

In [4]:
out_queue= ppq.TransferQueue().register_callback(print)

class Flow(ppc.AsyncFlow):

    def __init__(self):
        super().__init__()

        distributor = ppc.distributors.IteratorDistributor()
        compute3 = ComputeClass(1, 2)
        self.chain(
            (ppq.ThreadQueue(), distributor),
            (distributor, compute, ppq.ProcessQueue(5)),
                (compute, print, ppq.TransferQueue()),
                (compute, out_queue),
                (compute, compute2, ppq.ProcessQueue(5)),
                    (compute2, print),
                    (compute2, out_queue),
                    (compute2, compute3, ppq.ProcessQueue()),
                        (compute3, print),
                        (compute3, out_queue)
        )

flow = Flow()
flow

[ThreadQueue(name=None, runner=1, id=133618889030160)]

  |->(IteratorDistributor, id=133618889029488)

    |->[ProcessQueue(name=None, runner=5, id=133618889030496)]

      |->(<function compute at 0x79869258cb80>, id=133618887871360)

        |->[TransferQueue(name=None, runner=1, id=133618887649680)]

          |->(<built-in function print>, id=133625118581808)

        |->[TransferQueue(name=None, runner=1, id=133625070618336)]

        |->[ProcessQueue(name=None, runner=5, id=133618887651280)]

          |->(<function compute2 at 0x79869258cc20>, id=133618887871520)

            |->[TransferQueue(name=None, runner=1, id=133618887649680)]

              |->(<built-in function print>, id=133625118581808)...

            |->[TransferQueue(name=None, runner=1, id=133625070618336)]

            |->[ProcessQueue(name=None, runner=1, id=133618887651600)]

              |->(ComputeClass, id=133618889029824)

                |->[TransferQueue(name=None, runner=1, id=133618887649680)]

    

In [5]:
with flow:
   for i in range(10):
      flow.put([i] * 2)

compute: 0
compute: 0
compute: 1
compute: 2
compute: 1
compute: 2
compute: 3
compute: 3
compute2: compute: 0c
compute: 4
compute2: compute: 1c
compute: 4
compute2: compute: 2c
compute2: compute: 1c
compute2: compute: 0c
ComputeClass: compute2: compute: 0cd
ComputeClass: compute2: compute: 1cd
ComputeClass: compute2: compute: 2cd
ComputeClass: compute2: compute: 1cd
ComputeClass: compute2: compute: 0cd
compute: 5
compute: 5
compute: 6
compute2: compute: 2c
ComputeClass: compute2: compute: 2cd
compute: 6
compute2: compute: 3c
compute2: compute: 3c
compute: 7
ComputeClass: compute2: compute: 3cd
ComputeClass: compute2: compute: 3cd
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
compute: 8
compute: 7
compute: 8
compute2: compute: 5c
ComputeClass: compute2: compute: 5cd
compute2: compute: 5c
ComputeClass: compute2: compute: 5cd
compute: 9
compute: 9
ComputeClass: compute2: compute: 6cd
compute2: compute: 6c
compute2: com

In [6]:
out_queue.run()

compute: 0
compute: 0
compute: 1
compute: 2
compute: 1
compute: 2
compute: 3
compute: 3
compute2: compute: 0c
compute2: compute: 1c
compute: 4
compute: 4
compute2: compute: 2c
compute2: compute: 1c
compute2: compute: 0c
ComputeClass: compute2: compute: 0cd
ComputeClass: compute2: compute: 1cd
ComputeClass: compute2: compute: 2cd
ComputeClass: compute2: compute: 1cd
ComputeClass: compute2: compute: 0cd
compute: 5
compute: 5
compute: 6
compute2: compute: 2c
ComputeClass: compute2: compute: 2cd
compute: 6
compute: 7
compute2: compute: 3c
compute2: compute: 3c
ComputeClass: compute2: compute: 3cd
ComputeClass: compute2: compute: 3cd
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
compute: 7
compute: 8
compute: 8
compute2: compute: 5c
ComputeClass: compute2: compute: 5cd
compute: 9
compute: 9
ComputeClass: compute2: compute: 5cd
compute2: compute: 5c
compute2: compute: 6c
ComputeClass: compute2: compute: 6cd
compute2: com

TransferQueue(
  name=None,
  num_runner=1,
)

In [7]:
out_queue.release()